## Extra convenience methods for common responses

### OPeNDAP


In [1]:
from erddapy import ERDDAP

e = ERDDAP(server="https://gliders.ioos.us/erddap")

In [2]:
from netCDF4 import Dataset

e.constraints = None
e.protocol = "tabledap"
e.dataset_id = "whoi_406-20160902T1700"

opendap_url = e.get_download_url(
    response="opendap",
)

print(opendap_url)
with Dataset(opendap_url) as nc:
    print(nc.summary)

https://gliders.ioos.us/erddap/tabledap/whoi_406-20160902T1700
Slocum glider dataset gathered as part of the TEMPESTS (The Experiment to Measure and Predict East coast STorm Strength), funded by NOAA through CINAR (Cooperative Institute for the North Atlantic Region).


### netCDF Climate and Forecast


In [3]:
e.response = "nc"
e.variables = ["longitude", "latitude", "temperature", "salinity"]

nc = e.to_ncCF()

print(nc.Conventions)
print(nc["temperature"])

Unidata Dataset Discovery v1.0, COARDS, CF-1.10
<class 'netCDF4._netCDF4.Variable'>
float32 temperature(obs)
    _ChunkSizes: 106
    _FillValue: -999.0
    actual_range: [ 9.9473 22.6177]
    ancillary_variables: qartod_temperature_climatological_flag qartod_temperature_flat_line_flag qartod_temperature_gross_range_flag qartod_temperature_rate_of_change_flag qartod_temperature_spike_flag temperature_qc
    colorBarMaximum: 32.0
    colorBarMinimum: 0.0
    coordinates: time latitude longitude depth
    instrument: instrument_ctd
    ioos_category: Temperature
    long_name: Sea Water Temperature
    observation_type: measured
    platform: platform
    standard_name: sea_water_temperature
    units: Celsius
    valid_max: 40.0
    valid_min: -5.0
unlimited dimensions: 
current shape = (54245,)
filling on


### xarray


In [4]:
ds = e.to_xarray()

ds

<xarray.Dataset> Size: 672kB
Dimensions:          (trajectory: 1, profile: 525, obs: 54245)
Coordinates:
  * trajectory       (trajectory) object 8B 'whoi_406-20160902T1700'
    time             (profile) datetime64[ns] 4kB ...
    longitude        (profile) float64 4kB ...
    latitude         (profile) float64 4kB ...
    depth            (obs) float32 217kB ...
Dimensions without coordinates: profile, obs
Data variables:
    profile_id       (profile) float64 4kB ...
    trajectoryIndex  (profile) int32 2kB ...
    rowSize          (profile) int32 2kB ...
    temperature      (obs) float32 217kB ...
    salinity         (obs) float32 217kB ...
Attributes: (12/56)
    acknowledgment:                This deployment supported by NOAA through ...
    cdm_data_type:                 TrajectoryProfile
    cdm_profile_variables:         time_uv,lat_uv,lon_uv,u,v,profile_id,time,...
    cdm_trajectory_variables:      trajectory,wmo_id
    comment:                       Glider operatored by Woods Hole Oceanograp...
    contributor_name:              Robert Todd, Larry George, Patrick Deane, ...
    ...                            ...
    subsetVariables:               wmo_id,trajectory,profile_id,time,latitude...
    summary:                       Slocum glider dataset gathered as part of ...
    time_coverage_end:             2016-09-20T13:32:33Z
    time_coverage_start:           2016-09-02T17:05:38Z
    title:                         whoi_406-20160902T1700
    Westernmost_Easting:           -71.54982816987487

### iris


In [5]:
import warnings

# Iris warnings are quire verbose!
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    cubes = e.to_iris()

print(cubes)

0: sea_water_practical_salinity / (1)  (-- : 54245)
1: Profile ID / (unknown)              (-- : 525)
2: Number of Observations for this Profile / (unknown) (-- : 525)
3: The trajectory to which this profile is associated. / (unknown) (-- : 525)
4: sea_water_temperature / (Celsius)   (-- : 54245)
5: Trajectory Name / (unknown)         (-- : 1)
6: time / (seconds since 1970-01-01T00:00:00Z) (-- : 525)
7: longitude / (degrees)               (-- : 525)
8: latitude / (degrees)                (-- : 525)


In [6]:
cubes.extract_cube("sea_water_temperature")

<iris 'Cube' of sea_water_temperature / (Celsius) (-- : 54245)>